In [1]:
from keras import backend as K
K.set_image_dim_ordering('th')

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np
import pandas as pd

Using TensorFlow backend.
/Users/yanxilu/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from keras.models import Model

In [3]:
from matplotlib import pyplot as plt

In [4]:
from keras.applications.vgg16 import VGG16
from keras.applications import ResNet50
from keras.applications.imagenet_utils import decode_predictions
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
import tensorflow as tf
from skimage import exposure
from tensorflow import layers
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler

In [6]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Preprocess Images
def prepare_image(image_path):
    im = cv2.resize(cv2.imread(image_path), (224, 224)).astype(np.float32)

    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)
    return im

Training Model
There are four categories of the train_data (X_train): <br/>
Hoodie_Sweater <br/>
Shirt_Blouses <br/>
Jacket_Coat <br/>
T-Shirt

In [ ]:
#Data Generator for Validation Data
train, validation, labels, labelsvalidation = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
validationdatagenerator = ImageDataGenerator()
traindatagenerator = ImageDataGenerator(width_shift_range=0.2,height_shift_range=0.2,rotation_range=10,zoom_range=0.1 )
#Data Generator for Traning Data
train_generator=traindatagenerator.flow(train, labels, batch_size=batchsize) 
validation_generator=validationdatagenerator.flow(validation, labelsvalidation,batch_size=batchsize)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(X_train)

In [ ]:
#Architecture of Convolutional Neural Network

In [ ]:
model_resnet = ResNet50(weights = "imagenet")
model_resnet.layers.pop()
new_model = Sequential()
new_model.add(model_resnet)
for layer in new_model.layers[:-3]:
    layer.trainable = False
#Three more trainable layers are added. 
new_model.add(Dense(500,activation='relu'))
new_model.add(Dropout(0.1))
new_model.add(Dense(250,activation='relu'))
new_model.add(Dropout(0.1))
new_model.add(Dense(4))
new_model.add(Activation('softmax'))
new_model.compile(optimizer=optimizers.RMSprop(lr=0.0001),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Fitting the Model
new_model.fit_generator(train_generator, steps_per_epoch=int(len(train)/batchsize), epochs=30,verbose=1,validation_data=validation_generator, validation_steps=int(len(validation)/batchsize),)

In [ ]:
#Saving the Model
new_model.save_weights("new_model_weights.hdf5")

Extracting Feature Vector from the third to the last layers

In [ ]:
#Only the third to the last layer is extracted. 
embedded_features_model = Sequential()
embedded_features_model = Model(inputs=new_model.input, 
                                        outputs=new_model.get_layer(index=-3).output)
#Size of the feature vector.
embedded_features_model.output_shape
#Get feature vector and compare the similarity. 
test_v1_fc2 = embedded_features_model.predict(pic1)
test_v2_fc2 = embedded_features_model.predict(pic2)
cosine_similarity(test_v1_fc2,test_v2_fc2)